In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from skimage import filters
from skimage import morphology
import math
from skimage import measure
import time
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
dt=[('0' if x<10 else '')+str(x) for x in range(1, 15)]
dy=[[y+'_'+x for y in dt] for x in ['dr', 'h', 'g']]
z=[]
for x in dy:
    z=z+x
print(z)

flag=cv2.IMREAD_GRAYSCALE
#flag=cv2.IMREAD_COLOR
Manuals = [cv2.imread(f"manual1/{x}.tif", flag) for x in z]
Masks= [cv2.imread(f"mask/{x}_mask.tif", flag) for x in z]
Images= [cv2.imread(f"images/{x}.jpg", flag) for x in z]

Creation=list(zip(Manuals, Masks, Images))

"""for x in Images:
    #plt.imshow(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
    plt.imshow(x, cmap='gray')
    plt.show()
"""

['01_dr', '02_dr', '03_dr', '04_dr', '05_dr', '06_dr', '07_dr', '08_dr', '09_dr', '10_dr', '11_dr', '12_dr', '13_dr', '14_dr', '01_h', '02_h', '03_h', '04_h', '05_h', '06_h', '07_h', '08_h', '09_h', '10_h', '11_h', '12_h', '13_h', '14_h', '01_g', '02_g', '03_g', '04_g', '05_g', '06_g', '07_g', '08_g', '09_g', '10_g', '11_g', '12_g', '13_g', '14_g']


"for x in Images:\n    #plt.imshow(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))\n    plt.imshow(x, cmap='gray')\n    plt.show()"

In [212]:
def pad(img, con=3):
    dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
    dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    return dt

def getItem(img, x, y, con=3):
    huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    vz=[img[x-con:x+con,y-con:y+con].var()]
    col=[img[x,y]]
    lees=np.concatenate((huhu, col, vz))
    return lees

def getImagoData(img):
    Cnt=5
    dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    
    img2=pad(img, Cnt)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            dissolve[i*img.shape[0]+j]=getItem(img2, i+Cnt, j+Cnt, Cnt)
            
    return dissolve


def getImagoRes(bwimg):
    res=np.reshape(bwimg, (bwimg.shape[0]*bwimg.shape[1]))
    return res

def getImagoRandomoRes(resimg, tupls):
    res=[]
    for x in tupls:
        res.append(resimg[x[0], x[1]])
    return np.asarray(res)

#0 - manual, 1 - maska, 2 - oryginał
def randomizer(set_img):
    Z=500001
    Cnt=5
    img2=pad(set_img[2], Cnt)
    dissolve=[]
    tuplez=[]
    
    for i in range(Z):
        while (True):
            rn=np.random.randint(0, set_img[0].shape[0]*set_img[0].shape[1])
            rny=rn%set_img[1].shape[1]
            rnx=rn//set_img[1].shape[1]
                
            if (set_img[1][rnx,rny]!=0 and
                (set_img[0][rnx,rny]!=0 or
                 np.random.randint(0, 4)%4==0)):
                break
        dissolve.append(getItem(img2, rnx+Cnt, rny+Cnt, Cnt))
        tuplez.append((rnx, rny))
        #print(rnx, rny, set_img[0][rnx,rny])
    return np.asarray(dissolve), tuplez

In [213]:
t1=time.time()
x1, locs1=randomizer(Creation[0])
print(f'HUHU, {time.time()-t1}')
x2, locs2=randomizer(Creation[1])
t2=time.time()
print(t2-t1)


"""
for numb, x in enumerate(Creation):
    img=x[2].copy()
    t1=time.time()
    for i in range(img.shape[0]-Cnt):
        for j in range(img.shape[1]-Cnt):
            huhu=measure.moments_hu(img[i:i+Cnt,i:i+Cnt])
            zentral=measure.moments_central(img[i:i+Cnt,i:i+Cnt])
        print(i)
    t2=time.time()
    print(t2-t1)
"""

HUHU, 29.53576135635376
58.41581988334656


'\nfor numb, x in enumerate(Creation):\n    img=x[2].copy()\n    t1=time.time()\n    for i in range(img.shape[0]-Cnt):\n        for j in range(img.shape[1]-Cnt):\n            huhu=measure.moments_hu(img[i:i+Cnt,i:i+Cnt])\n            zentral=measure.moments_central(img[i:i+Cnt,i:i+Cnt])\n        print(i)\n    t2=time.time()\n    print(t2-t1)\n'

In [214]:
print(x1.shape)
y1=getImagoRandomoRes(Creation[0][0], locs1)
xp1=sklearn.preprocessing.normalize(x1)
xp2=sklearn.preprocessing.normalize(x2)

(500001, 9)


In [221]:
nigg=KNeighborsClassifier(n_neighbors=5)
t1=time.time()
nigg.fit(xp1, y1)
t2=time.time()
print(t2-t1)

2.0206305980682373


In [222]:
t1=time.time()
yp2=nigg.predict(xp2)
t2=time.time()
print(t2-t1)

print(len(yp2), len(y1))

11.891641855239868
500001 500001


In [223]:
def evaluator(im1, im2):
    dicc={}
    dicc["1st B"]=len(im1[im1==0])
    dicc["1st W"]=len(im1[im1!=0])
    dicc["2nd B"]=len(im2[im2==0])
    dicc["2nd W"]=len(im2[im2!=0])
    
    dicc["tp"]=len(im2[(im2==im1) & (im2!=0)])
    dicc["tn"]=len(im2[(im2==im1) & (im2==0)])
    dicc["fp"]=len(im2[(im2!=im1) & (im2!=0)])
    dicc["fn"]=len(im2[(im2!=im1) & (im2==0)])
    _=[print(x, dicc[x]) for x in ["tp", "tn", "fp", "fn"]]
    if (dicc["fn"]==0):
        dicc["fn"]=1
    
    dicc["acc"]=(dicc["tp"]+dicc["tn"])/(dicc["tp"]+dicc["tn"]+dicc["fp"]+dicc["fn"])
    dicc["prec"]=dicc["tp"]/(dicc["tp"]+dicc["fp"])
    dicc["rec"]=dicc["tp"]/(dicc["tp"]+dicc["fn"])
    
    return dicc

y2=getImagoRandomoRes(Creation[1][0], locs2)
dct=evaluator(y2, yp2)
print(f"acc:{dct['acc']}, prec:{dct['prec']}, rec:{dct['rec']}, geom:{math.sqrt(dct['prec']*dct['rec'])}")

tp 22961
tn 334384
fp 42152
fn 100504
acc:0.7146885706228587, prec:0.3526331147389922, rec:0.18597173287976348, geom:0.25608551583172146


In [175]:
print(sum(y2), sum(yp2))

1961205 2036175


In [31]:
np.asarray([[1,2,3],[4,5,6]]).var()

2.9166666666666665

In [ ]:
%%cython
import numpy as np
cimport numpy as np
from cpython cimport array
from skimage import measure
import array


def pad(img, con=3):
    dt=np.zeros((img.shape[0]+2*con, img.shape[1]+2*con), dtype='uint8')
    dt[con:dt.shape[0]-con,con:dt.shape[1]-con]=img
    return dt


def getItem(img, x, y, con=3):
    lees=np.asarray([], dtype='float')
    huhu=measure.moments_hu(img[x-con:x+con,y-con:y+con])
    lees=np.concatenate((lees, huhu))
    col=img[x,y]
    lees=np.concatenate((lees, np.asarray([col], dtype='float')))
    return lees

def getImagoData(unsigned char[:,:] img, int train=0):
    #cdef int s=0, t=0, gr=0, ij, i, jj
    cdef int i=0, j=0, Cnt=10, is1=img.shape[0], is2=img.shape[1], ij
    cdef np.ndarray[double, ndim=2] dissolve=np.zeros((img.shape[0]*img.shape[1], getItem(img, 200, 200).shape[0]))
    cdef np.ndarray[unsigned char, ndim=2] img2=pad(img, Cnt)
    cdef np.ndarray[double, ndim=1] tmp=np.zeros((getItem(img, 200, 200).shape[0]))
    
    for i in range(is1):
        for j in range(is2):
            tmp=getItem(img2, i+Cnt, j+Cnt, Cnt)
            for ij in range(tmp.shape[0]):
                dissolve[i*is1+j,ij]=tmp[ij]
    return dissolve

In [46]:
res=np.asarray([[1,2,3,4],[5,6,7,8]])
print(np.reshape(res, (res.shape[0]*res.shape[1])))



[1 2 3 4 5 6 7 8]
-3364981360.0
